### Task 1: Binary Representations

In [1]:
# Importing necessary module for 32-bit operations
import numpy as np

# ROTATE LEFT
def rotl(x, n=1):
    """Rotate left (circular shift) a 32-bit unsigned integer x by n bits."""
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# ROTATE RIGHT
def rotr(x, n=1):
    """Rotate right (circular shift) a 32-bit unsigned integer x by n bits."""
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

# CHOOSE FUNCTION
def ch(x, y, z):
    """
    For each bit position:
    - Choose bit from y if bit in x is 1
    - Choose bit from z if bit in x is 0
    """
    return (x & y) ^ (~x & z) & 0xFFFFFFFF

# MAJORITY FUNCTION
def maj(x, y, z):
    """
    For each bit position, return 1 if at least two of the bits in x, y, z are 1.
    """
    return ((x & y) ^ (x & z) ^ (y & z)) & 0xFFFFFFFF


### Task 2: Hash Functions

### Task 3: SHA256

### Task 4: Prime Numbers

### Task 5: Roots

### Task 6: Proof of work

### Task 7: Turing Machines

### Task 8: Computational Complexity